In [15]:
import numpy as np
import pandas as pd
np.random.seed(42)
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report



In [16]:
n = 1000

order_amount = np.random.uniform(100, 2000, n)
delay_minutes = np.random.randint(0, 90, n)
previous_refunds = np.random.randint(0, 5, n)
fraud_score = np.random.uniform(0, 1, n)
complaint_severity = np.random.randint(1, 6, n)

refund_prob = (
    0.3 * (delay_minutes > 30) +
    0.4 * (complaint_severity > 3) -
    0.5 * (fraud_score > 0.7)
)

refund_prob = 1 / (1 + np.exp(-refund_prob))
refunded = np.random.binomial(1, refund_prob)

data = pd.DataFrame({
    "order_amount": order_amount,
    "delay_minutes": delay_minutes,
    "previous_refunds": previous_refunds,
    "fraud_score": fraud_score,
    "complaint_severity": complaint_severity,
    "refunded": refunded
})

data.head()


,order_amount,delay_minutes,previous_refunds,fraud_score,complaint_severity,refunded
0,811.626226,46,4,0.057741,2,0
1,1906.357182,11,4,0.680756,5,1
2,1490.788489,61,1,0.466782,5,0
3,1237.451120,79,4,0.034461,2,1
4,396.435417,87,1,0.969608,5,0


In [17]:
data["rule_prediction"] = [
    rule_based_decision(v, d, p)
    for v, d, p in zip(
        data["order_amount"],
        data["delay_minutes"],
        data["previous_refunds"]
    )
]

rule_accuracy = calculate_accuracy(
    data["rule_prediction"],
    data["refunded"]
)

print("Rule-Based Accuracy:", rule_accuracy)


Rule-Based Accuracy: 0.548


In [18]:
X = data.drop(columns=["refunded", "rule_prediction"])
y = data["refunded"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

ml_predictions = model.predict(X_test)

ml_accuracy = accuracy_score(y_test, ml_predictions)

print("ML Accuracy:", ml_accuracy)
print("ML Confusion Matrix:")
print(confusion_matrix(y_test, ml_predictions))
print("\nML Classification Report:")
print(classification_report(y_test, ml_predictions))


ML Accuracy: 0.565
ML Confusion Matrix:
[[38 59]
 [28 75]]

ML Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.39      0.47        97
           1       0.56      0.73      0.63       103

    accuracy                           0.56       200
   macro avg       0.57      0.56      0.55       200
weighted avg       0.57      0.56      0.55       200



In [19]:
retention_value = 500
fraud_threshold = 0.7

def calculate_cost(df, predictions):
    total_cost = 0
    for i, pred in enumerate(predictions):
        row = df.iloc[i]
        if pred == 1:
            total_cost += row["order_amount"]
            if row["fraud_score"] > fraud_threshold:
                total_cost += row["order_amount"]
        else:
            if row["refunded"] == 1:
                total_cost += retention_value
    return total_cost

rule_cost = calculate_cost(data, data["rule_prediction"])
ml_cost = calculate_cost(data.loc[X_test.index], ml_predictions)

print("Total Economic Cost (Rule-Based):", rule_cost)
print("Total Economic Cost (ML-Based):", ml_cost)


Total Economic Cost (Rule-Based): 959841.1705491523
Total Economic Cost (ML-Based): 158277.1661673083


In [20]:
import sys
import os

sys.path.append(os.getcwd())

from rule_engine import rule_based_decision
from model import train_model, predict, predict_proba
from metrics import calculate_total_refund_cost, calculate_accuracy


In [21]:
from metrics import calculate_total_refund_cost

# Rule-based cost (full dataset)
rule_cost = calculate_total_refund_cost(
    data["rule_prediction"],
    data["order_amount"]
)

# ML cost (test set only)
ml_cost = calculate_total_refund_cost(
    ml_predictions,
    data.loc[X_test.index, "order_amount"]
)

print("Rule-Based Total Cost:", rule_cost)
print("ML-Based Total Cost:", ml_cost)


Rule-Based Total Cost: 668089.7202872157
ML-Based Total Cost: 132572.0132044251


In [22]:
from metrics import calculate_total_refund_cost


In [23]:
# Fair evaluation: compare both models on test set only

rule_test_cost = calculate_total_refund_cost(
    data.loc[X_test.index, "rule_prediction"],
    data.loc[X_test.index, "order_amount"]
)

ml_test_cost = calculate_total_refund_cost(
    ml_predictions,
    data.loc[X_test.index, "order_amount"]
)

print("Rule Test Cost:", rule_test_cost)
print("ML Test Cost:", ml_test_cost)


Rule Test Cost: 119911.30919960154
ML Test Cost: 132572.0132044251
